In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('/content/Train_aBjfeNk (1).csv')
test = pd.read_csv('/content/Test_LqhgPWU (1).csv')

###EDA and Preprocessing

In [ ]:
train.shape, test.shape

((8068, 11), (2627, 10))

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [ ]:
len(set(test['ID'].unique()).intersection(set(train['ID'].unique())))

2332

In [ ]:
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [ ]:
test.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,458996,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6
3,459000,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6
4,459001,Female,No,19,No,Marketing,NaN,Low,4.0,Cat_6


In [ ]:
train.isnull().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

In [ ]:
train['Segmentation'].value_counts()

D    2268
A    1972
C    1970
B    1858
Name: Segmentation, dtype: int64

In [ ]:
for i in ['Gender' , 'Ever_Married' , 'Graduated','Profession','Spending_Score','Var_1'  ]:
    print(f"column {i} unique values {train[i].unique()}")

column Gender unique values ['Male' 'Female']
column Ever_Married unique values ['No' 'Yes' nan]
column Graduated unique values ['No' 'Yes' nan]
column Profession unique values ['Healthcare' 'Engineer' 'Lawyer' 'Entertainment' 'Artist' 'Executive'
 'Doctor' 'Homemaker' 'Marketing' nan]
column Spending_Score unique values ['Low' 'Average' 'High']
column Var_1 unique values ['Cat_4' 'Cat_6' 'Cat_7' 'Cat_3' 'Cat_1' 'Cat_2' nan 'Cat_5']


In [ ]:
# concatenate data into df ,  fill missing values with mode & label encode category values
train['is_train'] = 1
test['is_train'] = 0
df = pd.concat([train,test])

In [ ]:
for column in df.columns:
  df[column].fillna(df[column].mode()[0], inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

for i in ['Gender' , 'Ever_Married' , 'Graduated','Profession','Spending_Score','Var_1']:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i].astype('str'))

In [ ]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation,is_train
0,462809,1,0,22,0,5,1.0,2,4.0,3,D,1
1,462643,0,1,38,1,2,1.0,0,3.0,3,A,1
2,466315,0,1,67,1,2,1.0,2,1.0,5,B,1
3,461735,1,1,67,1,7,0.0,1,2.0,5,B,1
4,462669,0,1,40,1,3,1.0,1,6.0,5,A,1


In [ ]:
df['Segmentation'] = df['Segmentation'].map({'A':0,'B':1,'C':2,'D':3})

In [ ]:
train = df[df['is_train'] == 1]

In [ ]:
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation,is_train
0,462809,1,0,22,0,5,1.0,2,4.0,3,3,1
1,462643,0,1,38,1,2,1.0,0,3.0,3,0,1
2,466315,0,1,67,1,2,1.0,2,1.0,5,1,1
3,461735,1,1,67,1,7,0.0,1,2.0,5,1,1
4,462669,0,1,40,1,3,1.0,1,6.0,5,0,1


###checking the model using cross validation approach

In [ ]:
!pip install catboost

     |████████████████████████████████| 65.9MB 57kB/s 


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [ ]:
X = train.drop(columns=['Segmentation', 'is_train', 'ID'], axis=1)
y = train['Segmentation']

In [ ]:
rf = RandomForestClassifier(n_estimators=50)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
acc=[]
for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  rf.fit(X_train, y_train)
  pred= rf.predict(X_test)
  acc_score = accuracy_score(y_test, pred)
  acc.append(acc_score)
  print(f"fold {fold} accuracy {acc_score}")

print(f"Mean accuracy score {np.mean(acc_score)}")

fold 0 accuracy 0.4801734820322181
fold 1 accuracy 0.48141263940520446
fold 2 accuracy 0.4677819083023544
fold 3 accuracy 0.4606323620582765
fold 4 accuracy 0.4885306881587105
Mean accuracy score 0.4885306881587105


###using catboost classifier

In [ ]:
categorical_features = ["Gender", "Ever_Married" ,"Graduated" ,"Profession" ,"Spending_Score" ,"Var_1" ]
rf2 = LGBMClassifier(learning_rate=0.02,
                    boosting_type='gbdt', max_depth=4,  objective='multiclass', 
                    random_state=100,  
                  n_estimators=1000 ,reg_alpha=0, reg_lambda=1, n_jobs=-1)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
acc=[]
for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  rf2.fit(X_train, y_train, categorical_feature = categorical_features)
  pred= rf2.predict(X_test)
  acc_score = accuracy_score(y_test, pred)
  acc.append(acc_score)
  print(f"fold {fold} accuracy {acc_score}")

print(f"Mean accuracy score {np.mean(acc_score)}")

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Ever_Married', 'Gender', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


fold 0 accuracy 0.5340768277571252
fold 1 accuracy 0.5390334572490706
fold 2 accuracy 0.5241635687732342
fold 3 accuracy 0.5269683818970862
fold 4 accuracy 0.5511469311841289
Mean accuracy score 0.5511469311841289


In [ ]:
test = df[df['is_train'] == 0][X.columns]
submission = pd.DataFrame(rf.predict(test), columns=['Segmentation'])